In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

import niezbędnych bibliotek

In [113]:
jokes = []

for i in range(1, 101):
    file = f'jokes/init{i}.html'
    with open(file, 'r') as file:
        html = file.read()
        soup = BeautifulSoup(html, features='html.parser')
        text = soup.extract().body.get_text().strip().replace('\n', ' ').replace('\t', ' ')
        jokes.append(text)

# odczyt pliku jester-data-1.xls
ratings = pd.read_excel('jester-data-1.xls', header=None)

odczytanie ocen oraz żartów z plików html i xls

In [114]:
ratings = ratings.iloc[:, 1:]

ratings = ratings.replace(99, float('nan'))

# obliczenie sredniej oceny dla kazdego zartu
ratings_mean = ratings.mean()

oczyszczenie danych oraz obliczenie średniej oceny dla każdego żartu

In [ ]:
model = SentenceTransformer('bert-base-cased')
embeddings = model.encode(jokes)

No sentence-transformers model found with name C:\Users\kubas/.cache\torch\sentence_transformers\bert-base-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\kubas/.cache\torch\sentence_transformers\bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

stworzenie embeddingów dla żartów

In [ ]:
embeddings.shape

In [ ]:
ratings_mean.shape

sprawdzenie rozmiaru embeddingów oraz ocen

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(embeddings, ratings_mean, test_size=0.2, random_state=3)

podział danych na zbiór treningowy oraz walidacyjny

In [ ]:
def learn(learning_rate=0.001, model_size=(100,), regularization=0.0):
    mlp = MLPRegressor(solver='sgd', alpha=regularization, learning_rate='constant',
                       learning_rate_init=learning_rate, hidden_layer_sizes=model_size)
    loss_train = []
    loss_val = []

    for epoch in range(500):
        mlp.partial_fit(X_train, y_train)

        pred_train = mlp.predict(X_train)
        loss_train.append(mean_squared_error(y_train, pred_train))

        pred_val = mlp.predict(X_val)
        loss_val.append(mean_squared_error(y_val, pred_val))

    return loss_train, loss_val

ustawiono 500 epok oraz stworzono funkcję learn, która zwraca stratę dla zbioru treningowego oraz walidacyjnego

In [ ]:
def show_plot(loss_train, loss_val):
    plt.plot(range(len(loss_train)), loss_train, label='Train Loss')
    plt.plot(range(len(loss_val)), loss_val, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.gca().set_facecolor('white')
    plt.gca().tick_params(colors='black')
    plt.show()

In [ ]:
loss_train, loss_val = learn()
show_plot(loss_train, loss_val)

jak widać po około 100 epokach model zaczyna się przeuczać gdyż na na danych treningowych straty maleją, a na danych walidacyjnych są na stabilnym poziomie

Wpływ tempa uczenia

In [ ]:
loss_train, loss_val = learn(learning_rate=0.0001)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(learning_rate=0.000001)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(learning_rate=0.005)
show_plot(loss_train, loss_val)

jak widać tempo uczenia wpływa na wyniki, gdyż przy zbyt małym tempie uczenia model uczy się zbyt wolno, a przy zbyt dużym zbyt szybko, przez co przeuczenie występuje dosyć szybko, ponadto przy najwyższym tempie uczenia widać pewne anomalie w postaci skoków straty

Wpływ rozmiaru modelu MLP

In [ ]:
loss_train, loss_val = learn(model_size=(10,))
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(model_size=(50,))
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(model_size=(100,100))
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(model_size=(300,300,300))
show_plot(loss_train, loss_val)

jak widać rozmiar modelu nie ma znaczącego wpływu na wyniki nauki, gdyż straty są na podobnym poziomie

Praktyczny test modelu

Wybrałem po 3 żarty, które są mało śmieszne ze strony https://inews.co.uk/light-relief/jokes/best-bad-jokes-137688
oraz 3 które mi się spodobały ze strony https://www.rd.com/list/funniest-jokes-of-all-time/  oraz 3 zaproponowane przez chata gpt , proces polegał na zamianie żartu w wektor a następnie wywołanie funkcji predict

In [ ]:
mlp = MLPRegressor(solver='sgd', alpha=0.0, learning_rate='constant',
                   learning_rate_init=0.001, hidden_layer_sizes=(100,), max_iter=5000)

mlp.fit(X_train, y_train)

In [ ]:
def rate_joke(joke):
    embedding = model.encode(joke)
    embedding = np.reshape(embedding, (1, -1))
    rating_prediction = mlp.predict(embedding)
    return print("Ocena", rating_prediction[0])

In [ ]:
bad_joke = ['What do you call a bear without any teeth? A gummy bear!']
rate_joke(bad_joke)

In [ ]:
bad_joke = ['Does anyone need an ark? I Noah guy!']
rate_joke(bad_joke)

In [ ]:
bad_joke = ['How do you make holy water? You boil the hell out of it.']
rate_joke(bad_joke)

In [ ]:
gpt_joke = ["Why did the scarecrow win an award? Because he was outstanding in his field!"]
rate_joke(gpt_joke)

In [ ]:
gpt_joke = ["What do you call a fish wearing a crown? King Neptune-tune!"]
rate_joke(gpt_joke)

In [ ]:
gpt_joke = ['Why don’t scientists trust atoms? Because they make up everything.']
rate_joke(gpt_joke)

In [ ]:
cool_joke = ["Why don’t pirates take a shower before they walk the plank? They just wash up on shore."]
rate_joke(cool_joke)

In [ ]:
cool_joke = ["You can’t believe everything you hear—but you can repeat it."]
rate_joke(cool_joke)

In [ ]:
cool_joke = ["Why the keyboard is always working? Because it has two shifts."]
rate_joke(cool_joke)

jak widać model ocenia żarty w sposób zgodny z moim odczuciem, gdyż żarty, które uważam za śmieszne otrzymują wysoką ocenę, a żarty, które uważam za mało śmieszne otrzymują niską ocenę lecz różnica jest bardzo niewielka, ponadto żarty zaproponowane przez chat gpt są oceniane przez sztuczną inteligencję jako najśmieszniejsze, wygląda mi to na bunt maszyn :)


Regularyzacja ma zapobiegać zbytniemu dopasowaniu się do danych treningowych, przez co model powinien lepiej generalizować, a straty na danych walidacyjnych powinny być mniejsze

In [ ]:
loss_train, loss_val = learn(regularization=0.001)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=0.01)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=0.1)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=1.0)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=10.0)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=50.0)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=100.0)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=200)
show_plot(loss_train, loss_val)

In [ ]:
loss_train, loss_val = learn(regularization=500.0)
show_plot(loss_train, loss_val)

Regularyzacja faktycznie spełnia swoje zadanie i zapobiega przeuczeniu, gdyż straty na danych walidacyjnych są mniejsze niż straty na danych treningowych, a straty na danych treningowych są większe niż straty bez regularyzacji, jednak dzieje się tak gdy parametr ten ustawimy na około 50 , przy niskich wartościach zmiany nie są zauważalne a przy wyższych zaczyna dochodzić do anomalii.